In [1]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer

import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [2]:
import pickle

In [3]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

In [4]:
text_example = data[0]["text1"]

In [5]:
text_example = text_example.replace(". ", ".\n")

In [6]:
print(text_example)

Unbekannte griffen in der vergangenen Nacht einen Kiosk in Kreuzberg an.
Gegen 0.40 Uhr bemerkten Zeugen eine Gruppe von ungefähr sieben Männern, die mehrere Steine gegen das Schaufenster des Geschäftes an der Graefestraße warfen und die Scheibe beschädigten.
Anschließend schütteten die Angreifer eine Flüssigkeit gegen das Fenster und entzündeten sie.
Die anschließenden Flammen sollen bis in das erste Obergeschoss gereicht haben, wurden jedoch von Anwohnern gelöscht.
Die Tatverdächtigen flüchteten unerkannt in Richtung Hasenheide.
Die Ermittlungen führt ein Brandkommissariat beim Landeskriminalamt Berlin.


In [11]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt
SVP
MO
UFF
UFF
UFF
MO


In [13]:
for g,tree in gs: 
    for prop in g.getPropositions('pdf'):
        curProp = '{0}:({1})'.format(prop.pred,
                                      ", ".join([rel + ":" + arg for rel,arg in sorted(prop.args,key=lambda rel:prop.rel_order(rel[0]))]))
        print(curProp)

angreifen:(subj:Unbekannte , dobj:einen Kiosk in Kreuzberg , prep_in:der vergangenen Nacht )
bemerkt:(subj:Zeugen , dobj:eine Gruppe von ungefähr sieben Männern , die mehrere Steine gegen das Schaufenster des Geschäftes an der Graefestraße warfen und die Scheibe beschädigten , prep_gegen:00:40 Uhr )
werfen:(subj:ungefähr sieben Männern , und die Scheibe beschädigten , dobj:mehrere Steine , prep_gegen:das Schaufenster des Geschäftes an der Graefestraße )
entzünden:(subj:die Angreifer , dobj:sie )
Anschließend schütten:(subj:die Angreifer , dobj:eine Flüssigkeit gegen das Fenster )
und:(conj_und:Anschließend schütteten die Angreifer eine Flüssigkeit gegen das Fenster , conj_und:die Angreifer entzündeten sie )
reichen:(subj:Die anschließenden Flammen , prep_in:das erste Obergeschoss )
unerkannt flüchten:(subj:Die Tatverdächtigen , prep_in:Richtung Hasenheide )
führen:(subj:ein Brandkommissariat , dobj:Die Ermittlungen , prep_bei:Landeskriminalamt Berlin )


In [14]:
prop = g.getPropositions('pdf')[0]
print(prop.args[0][1])

Die Ermittlungen 
